In [33]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

In [2]:
file = open("C:\\Users\\MGC\\Downloads\\question9\\train.csv (3)\\train.csv", encoding='utf-8')
n = sum(1 for line in file) - 1
s = 200000
skip = sorted(random.sample(range(1, n+1), n-s))
chunk = pd.read_csv("C:\\Users\\MGC\\Downloads\\question9\\train.csv (3)\\train.csv", low_memory=False, chunksize=10000, skiprows=skip, )
df = pd.concat(chunk)

In [3]:
df

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,000079a261a28c51c4ae8393c7fd97e8,win8defender,1.1.15000.2,4.10.14393.1593,1.271.613.0,0,7.0,0,NaN,53447.0,...,14728.0,0,0.0,0.0,0,0,0.0,0.0,1.0,0
1,0000f4908a3eeda3b9b32b87b74b23cc,win8defender,1.1.14800.3,4.14.17639.18041,1.267.711.0,0,7.0,0,NaN,59792.0,...,13299.0,0,NaN,1.0,0,0,0.0,0.0,3.0,0
2,00011eb8bde7bc39ffc4cedc57f34f35,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1503.0,0,7.0,0,NaN,53447.0,...,12416.0,0,0.0,0.0,0,0,1.0,1.0,11.0,1
3,0001207d23d1b1d274bfae1683b3b2af,win8defender,1.1.15100.1,4.10.209.0,1.273.1379.0,0,7.0,0,NaN,53447.0,...,63175.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
4,000164411a99ebd6935520072674c8b4,win8defender,1.1.14800.3,4.14.17613.18039,1.267.359.0,0,7.0,0,NaN,53447.0,...,5493.0,0,0.0,0.0,0,0,0.0,0.0,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,fffec4b3fda7b44e5049d055e64917bd,win8defender,1.1.14901.4,4.8.10240.17443,1.269.396.0,0,7.0,0,NaN,12202.0,...,38697.0,0,0.0,0.0,0,0,0.0,1.0,11.0,1
199996,fffeebeeb3d4c7572fd99c46e7836821,win8defender,1.1.15100.1,4.12.16299.15,1.273.1601.0,0,7.0,0,239.0,53447.0,...,16549.0,0,NaN,0.0,0,0,0.0,0.0,7.0,0
199997,fffef18213e2469a7953ae1796112280,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1490.0,0,7.0,0,NaN,53447.0,...,7360.0,1,NaN,0.0,0,0,0.0,0.0,15.0,1
199998,ffff1b5dc9fbd2a903b79cf3a5357a78,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1668.0,0,0.0,1,NaN,3371.0,...,33154.0,1,NaN,0.0,0,0,0.0,1.0,10.0,1


In [4]:
df.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,200000.000000,199303.000000,200000.000000,9727.000000,199145.000000,199145.000000,199145.000000,200000.000000,200000.000000,192833.000000,...,196445.000000,200000.000000,73459.0,199665.000000,200000.000000,200000.000000,198358.000000,193278.000000,193278.000000,200000.000000
mean,0.000005,6.842441,0.017605,1665.600288,47810.958422,1.327485,1.021296,0.988075,107.898545,81358.536775,...,32993.905653,0.486685,0.0,0.007312,0.124870,0.037520,0.057053,0.283017,7.887147,0.500415
std,0.002236,1.034600,0.131511,993.128407,14047.842303,0.523977,0.167851,0.108549,62.960819,48922.430633,...,21195.210614,0.499824,0.0,0.085199,0.330572,0.190033,0.231945,0.450466,4.555043,0.500001
min,0.000000,0.000000,0.000000,1.000000,16.000000,1.000000,0.000000,0.000000,1.000000,7.000000,...,33.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,7.000000,0.000000,788.000000,49480.000000,1.000000,1.000000,1.000000,51.000000,36825.000000,...,13156.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.000000,7.000000,0.000000,1632.000000,53447.000000,1.000000,1.000000,1.000000,97.000000,82373.000000,...,33070.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000
75%,0.000000,7.000000,0.000000,2388.000000,53447.000000,2.000000,1.000000,1.000000,161.250000,123700.000000,...,52402.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,1.000000
max,1.000000,8.000000,1.000000,3204.000000,70486.000000,5.000000,4.000000,1.000000,222.000000,167953.000000,...,72102.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 83 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   MachineIdentifier                                  200000 non-null  object 
 1   ProductName                                        200000 non-null  object 
 2   EngineVersion                                      200000 non-null  object 
 3   AppVersion                                         200000 non-null  object 
 4   AvSigVersion                                       200000 non-null  object 
 5   IsBeta                                             200000 non-null  int64  
 6   RtpStateBitfield                                   199303 non-null  float64
 7   IsSxsPassiveMode                                   200000 non-null  int64  
 8   DefaultBrowsersIdentifier                          9727 non-null    float6

### Dropping some unneccessary columns

In [6]:
df.dropna(axis=1, thresh=120000, inplace=True)

In [7]:
correlation_matrix = df.corr()
threshold = 0.8 
highly_correlated_columns = set()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            # Adding both columns to the set of highly correlated columns
            col_i = correlation_matrix.columns[i]
            col_j = correlation_matrix.columns[j]
            highly_correlated_columns.add(col_i)
            highly_correlated_columns.add(col_j)


In [8]:
df = df.drop(columns=highly_correlated_columns)

### Label encoding

In [9]:
le = LabelEncoder()

In [10]:
for i in df.columns:
    if df[i].dtype == "object":
        df[i] = le.fit_transform(df[i])        

### Splitting the dataset

In [22]:
df.replace(np.nan, 0, inplace=True)

In [23]:
X = df.drop("HasDetections", axis=1)
y = df['HasDetections']

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Training the data and evaluation

In [42]:
model = lgb.LGBMClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_val)

# Calculating accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6436


In [43]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.65      0.65     20043
           1       0.64      0.64      0.64     19957

    accuracy                           0.64     40000
   macro avg       0.64      0.64      0.64     40000
weighted avg       0.64      0.64      0.64     40000

